In [34]:
import pandas as pd
import numpy as np
#Problem 1
# Thêm header vào DataFrame để diễn giải dữ liệu
column_names = ['ID', 'Name', 'Age', 'Weight', 'm0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']

# Read the CSV file
df = pd.read_csv(r'/home/patient_heart_rate.csv', names=column_names, on_bad_lines='skip', skiprows=1)

# Display the first few rows of the dataframe
print("Problem 1:")
print(df.head(10))

#Problem 2
df[['FirstName', 'LastName']] = df['Name'].str.split(expand=True)
df= df.drop('Name', axis=1)
print("Problem 2:")
print(df.head(10))
'''
#Problem 3
#Get the Weight column
weight = df["Weight"]

def convert_weight(weight):
    if isinstance(weight, str):
        if 'lbs' in weight:
            weight = weight[:-3]
            return str(int(float(weight) / 2.2)) + 'kgs'
    return weight

# Áp dụng hàm chuyển đổi cho cột 'Weight' trên bản sao DataFrame
df["Weight"] = weight.apply(convert_weight)
print("Problem 3:")
print(df.head(10))

#Problem 4
df.dropna(how='all', inplace=True)
print("Problem 4:")
print(df.head(10))

#Problem 5
df=df.drop_duplicates(subset=["FirstName","LastName","Age","Weight"])
print("Problem 5:")
print(df.head(10))

#Problem 6
df.FirstName.replace({r'[\x00-\x7F]+':''}, regex=True, inplace=True)
df.LastName.replace({r'[\x00-\x7F]+':''}, regex=True, inplace=True)
print("Problem 6:")
print(df.head(10))


#Problem 7
#Deletion: Remove records with missing values
df = df.dropna(subset=['Age', 'Weight'])
#print("\nDataFrame after Deletion:")
#print(df.head())
#Dummy substitution: Replace missing values with a dummy but valid value:e.g.: 0 for numerical values
df_dummy = df.fillna({'Age': 0, 'Weight': '0kgs'})
#print("\nDataFrame after Dummy substitution:")
#print(df_dummy.head())
#Mean substitution: Replace the missing values with the mean
df_mean = df.copy()
df_mean['Age'] = df_mean['Age'].fillna(df_mean['Age'].mean())
df_mean['Weight'] = df_mean['Weight'].fillna(df_mean['Weight'].apply(lambda x: int(x[:-3]) if isinstance(x, str) else x).mean())
df_mean['Weight'] = df_mean['Weight'].astype(str) + 'kgs'
#print("\nDataFrame after Mean substitution:")
#print(df_mean.head())
#Frequent substitution: Replace the missing values with the most frequent item.
df_frequent = df.copy()
df_frequent['Age'] = df_frequent['Age'].fillna(df_frequent['Age'].mode()[0])
df_frequent['Weight'] = df_frequent['Weight'].fillna(df_frequent['Weight'].mode()[0])
#print("\nDataFrame after Frequent substitution:")
#print(df_frequent.head())
#Yêu cầu
# Thống kê thông tin dữ liệu thiếu
missing_info = df[['Age', 'Weight']].isnull().sum()
missing_info_percentage = df[['Age', 'Weight']].isnull().mean() * 100
print("Problem 7:")
print("Missing values count:\n", missing_info)
print("Missing values percentage:\n", missing_info_percentage)

#Problem 8
#Mekt the Sex+ time range column in single column
df = pd.melt(df, id_vars=['ID','Age','Weight', 'FirstName', 'LastName'], var_name='sex_and_time').sort_values(['ID','Age','Weight','FirstName','LastName'])
#Extract Sex, Hour Lower bound and Hour upper bound group
tmp_df = df["sex_and_time"].str.extract(r"(\D)(\d+)(\d{2})", expand=True) # Added opening parenthesis before \D to balance the expression
#Name columns
tmp_df.columns = ['Sex', 'Hour_Lower', 'Hour_Upper']
#Create Time column based in 'Hour_Lower' and "hour_Upper" columns
tmp_df["Time"]=tmp_df["Hour_Lower"]+ "-" +tmp_df["Hour_Upper"]
#merge
df = pd.concat([df, tmp_df], axis=1)
#Drop unneccessary columns and rows
df = df.drop(['sex_and_time','Hour_Lower','Hour_Upper'], axis=1)
df = df.dropna()
df.to_csv('outputcleanup.csv', index=False)
print("Problem 8:")
print(df.head(10))


#Problem 9

# Calculate the missing data percentage on blood pressure columns

## Calculate the mean of the "Age" column
age_mean = df['Age'].astype(float).mean()

# Replace missing values in "Age" with the mean
df['Age'] = df['Age'].fillna(age_mean)

# Remove rows where both "Age" and "Weight" are missing
df.dropna(subset=['Age', 'Weight'], how='all', inplace=True)
'''
# Ensure heart rate columns are numeric
heart_rate_columns = ["m0006", "m0612", "m1218", "f0006", "f0612", "f1218"]
df[heart_rate_columns] = df[heart_rate_columns].apply(pd.to_numeric, errors='coerce')

# Define a function to fill missing values in heart rate columns
def fill_heart_rate_missing_values(df):
    for col in heart_rate_columns:
        for i in range(len(df)):
            if pd.isnull(df.loc[i, col]):
                prev_val = df.loc[i - 1, col] if i - 1 >= 0 else np.nan
                next_val = df.loc[i + 1, col] if i + 1 < len(df) else np.nan

                if not pd.isnull(prev_val) and not pd.isnull(next_val):
                    df.loc[i, col] = (prev_val + next_val) / 2
                else:
                    prev_two_vals = df.loc[i - 2:i - 1, col].mean() if i - 2 >= 0 else np.nan
                    next_two_vals = df.loc[i + 1:i + 2, col].mean() if i + 2 < len(df) else np.nan
                    individual_mean = df.loc[:, heart_rate_columns].mean(axis=1).iloc[i]

                    if not pd.isnull(df.loc[i, 'FirstName']):
                        gender_mean = df[df['FirstName'].str[0] == df.loc[i, 'FirstName'][0]][heart_rate_columns].mean().mean()
                    else:
                        gender_mean = np.nan

                    overall_mean = df[heart_rate_columns].mean().mean()

                    if not pd.isnull(prev_two_vals):
                        df.loc[i, col] = prev_two_vals
                    elif not pd.isnull(next_two_vals):
                        df.loc[i, col] = next_two_vals
                    elif not pd.isnull(individual_mean):
                        df.loc[i, col] = individual_mean
                    elif not pd.isnull(gender_mean):
                        df.loc[i, col] = gender_mean
                    elif not pd.isnull(overall_mean):
                        df.loc[i, col] = overall_mean
                    else:
                        df.loc[i, col] = 60

    return df

# Apply the function to fill missing values
df = fill_heart_rate_missing_values(df)

# Display the dataframe after replacing missing blood pressure values
print("Problem 9:")
print(df.head(10))
print("\nDataFrame after replacing missing blood pressure values:")
print(df.head())
# Problem 10

# Drop any rows with missing values in the dataset
df_clean = df.dropna()

# Reindex the cleaned dataframe
df_clean.reset_index(drop=True, inplace=True)

# Save the cleaned data to a CSV file
df_clean.to_csv(r'patient_heart_rate_clean.csv', index=False)

print("Problem 10:")
print(df_clean.head(10))
print("Cleaned data saved to patient_data_clean.csv successfully.")







Problem 1:
    ID            Name   Age      Weight m0006 m0612 m1218 f0006 f0612 f1218
0  2.0     Donald Duck  34.0   154.89lbs     -     -     -    85    84    76
1  3.0      Mini Mouse  16.0         NaN     -     -     -    65    69    72
2  4.0  Scrooge McDuck   NaN       78kgs    78    79    72     -     -     -
3  5.0    Pink Panther  54.0  198.658lbs     -     -     -    69   NaN    75
4  6.0     Huey McDuck  52.0      189lbs     -     -     -    68    75    72
5  7.0    Dewey McDuck  19.0       56kgs     -     -     -    71    78    75
6  8.0      Scööpy Doo  32.0       78kgs    78    76    75     -     -     -
7  NaN             NaN   NaN         NaN   NaN   NaN   NaN   NaN   NaN   NaN
8  NaN             NaN   NaN         NaN   NaN   NaN   NaN   NaN   NaN   NaN
9  9.0     Huey McDuck  52.0      189lbs     -     -     -    68    75    72
Problem 2:
    ID   Age      Weight m0006 m0612 m1218 f0006 f0612 f1218 FirstName  \
0  2.0  34.0   154.89lbs     -     -     -    85    84   